In [2]:
import autograd.numpy as np
import scipy.optimize as opt
import scipy as sci
import random
import matplotlib.pyplot as plt
from autograd import grad, jacobian
import sys
sys.path.append("../")
#from ddn.basic.learnable_robust_nodes import LearnableRobustAverage
#from ddn.basic.robust_nodes import RobustAverage

# 1. Find the analytic gradient of alpha in robust pooling (Dy($\alpha$))
I have the detail calculation available

(a) pseudo-huber

$y \in  \text{argmin}_y  \sum_i^n \alpha^2 (\sqrt {1+ \frac {(y-x_i)^2} {\alpha^2}}-1)$


$Dy(\alpha) = -\sum_{i=1}^N \sum_{j=1}^N((\frac {y-x_i} {\alpha})^2 + 1)^{\frac{3}{2}}(\frac {(y-x_j)^3} {((\frac {y-x_j} {\alpha})^2 + 1)^{\frac{3}{2}}\alpha^3})$

(b) huber

$y \in \text{argmin}_y \sum_{i=1}^N $ $\begin{cases} 
\frac {1} {2} (y-x_i)^2 & \text{$ |y-x_i| \leq \alpha $} \\
\alpha(|y-x_i|-\frac {1} {2} \alpha) & \text{otherwise}\\
\end{cases}$

$Dy(\alpha) = \sum_{i=1}^N $ $\begin{cases} 
0 & \text{$ |y-x_i| \leq \alpha $} \\
1 & \text{$ y - x_i > \alpha $}\\
-1 & \text{$ y - x_i < \alpha $}
\end{cases}$

(c) welsch

$y \in  \text{argmin}_y  \sum_{i=1}^n (1-exp(- \frac {(y-x_i)^2} {2\alpha^2}))$

$Dy(\alpha) = \sum_{i=1}^N (- \frac {e^{-\frac {(y-x_i)^2} {2 \alpha^2}} (2 (y-x_i) \alpha^2 - (y-x_i)^3)} {\alpha^5}) $

(d) Truncated Quadatic(Q: Does it make sense to train $\alpha$ is this case)

$y \in \text{argmin}_y \sum_{i=1}^N $ $\begin{cases} 
\frac {1} {2} (y-x_i)^2 & \text{$ |y-x_i| \leq \alpha $} \\
\frac {1} {2} \alpha^2 & \text{otherwise}\\
\end{cases}$

$Dy(\alpha) = 0$

# 2. Implment gradient on robust pooling( fix x)
Implement analyical gradient of $\alpha$, check the correctness by autograd gradient

In [3]:
# example of learnable robust pooling('pseudo-huber')
def f(x, y, alpha,p='pseudo-huber'):
    alpha_sq=alpha**2
    if p=='pseudo-huber':
        phi= lambda z: (alpha**2) * (np.sqrt(1.0 + np.power(z, 2.0) / (alpha**2)) - 1.0)
    elif p=='huber':
        phi = lambda z: np.where(np.abs(z) <= alpha, 0.5 * np.power(z, 2.0), alpha * np.abs(z) - 0.5 * alpha_sq)
    elif p=='welsch':
        phi = lambda z: 1.0 - np.exp(-0.5 * np.power(z, 2.0) / alpha_sq)
    return np.sum([phi(y - xi) for xi in x])

def solve(x,alpha ,f, p='pseudo-huber'):
    result = opt.minimize(lambda y : f(x, y, alpha,p), np.mean(x))
    return result.x

def dyalpha_closed_form(x,y,alpha,p='pseudo-huber'):
    alpha_sq=alpha**2

    if p=='pseudo-huber':
        dyy = np.array([np.power(1.0 + np.power(y - xi, 2.0) / alpha_sq, -1.5) for xi in x])
        dytheta =  np.sum([np.power(y-xi,3)/(np.power(np.power((y-xi)/alpha,2)+1,1.5)*np.power(alpha,3)) for xi in x])
    elif p=='huber':
        dyy = np.array([1.0 if np.abs(y - xi) <= alpha else 0.0 for xi in x])
        dytheta = np.sum(np.array([0.0 if np.abs(y - xi) <= alpha else (1.0 if y-xi>0 else -1.0) for xi in x]))
    elif p=='welsch':
        z = np.power(x - y, 2.0)
        dyy = np.array([(alpha_sq - zi) / (alpha_sq * alpha_sq) * np.exp(-0.5 * zi / alpha_sq) for zi in z])
        dytheta=np.sum(np.array([-np.exp(-0.5 * np.power((y - xi)/alpha,2))*((2*(y-xi)*alpha_sq-np.power(y-xi,3))/(alpha**5)) for xi in x])) 
    return -1.0 * dytheta/np.sum(dyy)
    

def dyalpha(x,y,alpha,p='pseudo-huber'):
    fY = grad(f, 1)
    fYY = jacobian(fY, 1)
    fthetaY = jacobian(fY, 2)
    return -1.0 * np.linalg.pinv(fYY(x, y, alpha,p)).dot(fthetaY(x, y, alpha,p))

In [4]:
p='huber'
n = 10 # number of input points
y_target = np.array([0.0])
x_init = np.random.rand(n)
# add an outlier
x_init[np.random.randint(len(x_init))] += 100.0 * np.random.rand(1)
alpha_init = 1.0
y_init = solve(x_init,alpha_init,f,p)
# valid the analyic gradient is the same as autograd solution
print(dyalpha_closed_form(x_init,y_init,alpha_init,p)-dyalpha(x_init,y_init,alpha_init,p))

[0.]


In [5]:

def euler_method(init_alpha, step_size=1/100,p='pseudo-huber'):
    y=y_init
    alpha = init_alpha
    alphas=[]
    dalphas=[]
    for i in range(100):
        dalpha=dyalpha_closed_form(x_init,y_init,1.0-i/100)
        dalphas.append(dalpha)
        if dyalpha_closed_form(x_init,y,alpha)-dyalpha(x_init,y,alpha)>1**(-15):
            print("error in gradient calculation")
        new_alpha=alpha+step_size*dalpha
        y=solve(x_init,new_alpha,f,p) 
        alpha=new_alpha
        alphas.append(alpha)
    return alphas,dalphas
alphas,dalphas=euler_method(1.0, 1/100,p)
#plt.figure()
#plt.plot(range(100),alphas)
#plt.figure()
#plt.plot(range(100),dalphas)

#  Q
Q1: add constraint on alpha(sometimes alpha goes into negative)? eg. 1> alpha>0, so it is not unconstrainted node now.  

Q2: b'ABNORMAL_TERMINATION_IN_LNSRCH'

Q3: case when alpha did not converage to zero(counter intuition): someone 

Sometimes gradient descent stuck on saddle points

Sometimes it is indeed the minimum


for Huber:

[3.42539726e-02 7.72143347e-01 1.04526553e-01 9.21828824e-01
 1.83865213e-01 8.95758725e-01 3.02897183e-01 7.75589711e-01
 5.75906484e+01 1.98591063e-01]
 
Pseudo-Huber: 
[ 0.76438634  0.24721799  0.64154246  0.32604462 32.22827217  0.48242317
  0.0484155   0.92684649  0.71640308  0.54642665]
  
  
Q4: convex of f($\alpha$) and y($\alpha$)
 


In [6]:
# copy and modified from tutorial 2.
def simpleGradientDescent(node, y_target, step_size=1.0e-3, tol=1.0e-7, max_iters=100, x_init=None, verbose=False):
    """
    An example of gradient descent for a simple bi-level optimization problem of the form:
        minimize_{x} 0.5 * \|y - y_target\|^2
        subject to y = argmin_u f(x, u) s.t. h(x, u) = 0

    Returns the solution x found and learning curve (objective function J per iteration).
    """
    assert y_target.shape[0] == node.dim_y
    x = x_init.copy() if x_init is not None else np.zeros((node.dim_x,))

    J = lambda y : 0.5 * np.sum(np.square(y - y_target))
    dJdy = lambda y : y - y_target
    
    theta= node.get_alpha()
    thetas=[theta]
    history = []
    for i in range(max_iters):
        # solve the lower-level problem and compute the upper-level objective
        y, _ = node.solve(x)
        history.append(J(y))
        if verbose: print("{:5d}: {}".format(i, history[-1]))
        if (len(history) > 2) and (history[-2] - history[-1]) < tol:
            break
        # compute the gradient of the upper-level objective with respect to x via the chain rule
        _,dytheta= node.gradient(x, y)
        dJdtheta = (dJdy(y)* dytheta)[0]
        theta -=step_size * dJdtheta
        node.update_alpha(theta)
        thetas.append(theta)
        # take a step in the negative gradient direction
    return x, history, thetas

def lbfgs(node, y_target, max_iters=1000, x_init=None, verbose=True):
    """
    Example of using scipy.optimize to solve the problem via L-BFGS.
    """
    assert y_target.shape[0] == node.dim_y
    x_start = x_init.copy() if x_init is not None else np.zeros((node.dim_x,))
    theta=node.get_alpha()
    def J(theta):
        y, _ = node.solve(x_init, theta)
        return 0.5 * np.sum(np.square(y - y_target))
    def dJdtheta(theta):
        y, _ = node.solve(x_init, theta)
        return (y - y_target)* node.gradient(x_init, y, theta)[1]
    history = [J(theta)]
    def callback(theta):
        node.update_alpha(theta)
        history.append(J(theta))

    opts = {'maxiter': max_iters, 'disp': verbose}
    result = opt.minimize(J, np.mean(x_init), args=(), method='L-BFGS-B', options=opts, callback=callback)
    return result.x, history

In [7]:
n = 10 # number of input points
y_target = np.array([0.0])
x_init = np.random.rand(n)
# add an outlier
x_init[np.random.randint(len(x_init))] += 100.0 * np.random.rand(1)


# x_bfgs, history_bfgs = lbfgs(node, y_target, x_init=x_init)
# print(history_bfgs )

In [8]:
node = LearnableRobustAverage(n, penalty='huber', alpha=1.0)
x_gd, history,thetas = simpleGradientDescent(node, y_target, step_size=0.5, x_init=x_init)
print(x_init)

NameError: name 'LearnableRobustAverage' is not defined

In [ ]:
plt.figure()
plt.plot(thetas)
plt.figure()
plt.plot(history)

In [ ]:
print(history[-1])
print(thetas[-1])

In [ ]:
J = lambda y : 0.5 * np.sum(np.square(y - y_target))
nodes = [RobustAverage(n, penalty='huber', alpha=a) for a in np.linspace(0.0, 1.0, num=100)[1:]]
solvesd= [J(n.solve(x_init)[0]) for n in nodes]
plt.plot(solvesd)

# L2 Ball projection learnable radius.
## 1. analyical gradient using DDN theorm with equality constraint.
$$
\begin{array}{lll}
    y \in & \text{argmin}_u & \frac{1}{2} \|u - x\|^2 \\
    & \text{subject to} & u_1^2 + u_2^2 - r^2 = 0 \\
    & & 
\end{array}
$$

$$
\begin{array}{llll}
    A = \text{D}_{Y} h(r,y) &= 2 u^T \\
    B = \text{D}^2_{rY} f(r, y) - \sum_{i=1}^{3} \lambda_i \text{D}^2_{rY} h_i(r,y) &=0 \\
    C = \text{D}_{Y} h(r,y) &= -2r\\
    2u^T \lambda= u^T -x^T \Rightarrow \lambda= \sqrt{\frac {(u^T-x^T)(u-x)} {4^T}}\\ 
    H = \text{D}^2_{YY} f(r, y) - \sum_{i=1}^{3} \lambda_i \text{D}^2_{YY} h_i(r,y) &= (1 - 2 \lambda_1) I \\
    Dy(r) =  r \cdot \frac {y} {y^T y}
\end{array}
$$

##  2. analyical gradient of closed form.
$$
\begin{array}{lll}
    y(x,r) = \frac {r} {\| x\|_2} x\\
    Dy(r) = \frac {1} {\| x\|_2} x\\
\end{array}
$$


## 3.  gradient using DDN theorm with autograd.
$$
\begin{array}{lll}
    Dy(r) = (H^{-1} A^T(AH^{-1}A^T)^{-1}AH^{-1}B-C)-H^{-1}B
\end{array}
$$

Below we can verify three approaches gives the same solution.

\begin{array}{lll}
    y \in & \text{argmin}_u & \frac{1}{2} \|u - x\|^2_2 \\
    & \text{subject to} & \|x\|_p \leq r \\
\end{array}

\begin{array}{lll}
    y \in & \text{argmin}_u & \frac{1}{2} \|u - x\|^2_2 \\
    & \text{subject to} & \max_i |x_i| = r \\
    & a = vec(sign(y))^T\\
    & B = 0 \\
    & H = I \\
    & C = -1 \\
\end{array}


In [ ]:

x_init = np.random.rand(2)
r_init = random.uniform(0.1, 10)
x=x_init
r=random.uniform(0.1, 10)
def f(r,y,x):
    return 0.5* np.dot(y-x,y-x)
    
def h(r,y,norm='L2'):
    if norm=='L1':
        return np.linalg.norm(y, 1)**2 - r**2
    if norm=='Ln':
        return np.linalg.norm(y, np.inf)**2 - r**2
    else:
        return np.dot(y,y) - r**2

def solve_opt(x,r,f):
    result = opt.minimize(lambda y: f(r, y ,x), np.array([1.0,1.0]),constraints=[{'type':'eq', 'fun': lambda y: h(r,y)}] )
    return result.x

def solve_analyical(x,r):
    return r / np.sqrt(np.dot(x, x)) * x

def gradient_closed_form(r,y,x):
    return 1 / np.sqrt(np.dot(x, x)) * x

def gradient_analyic(r,x):
    y = solve_analyical(x,r)
    # a = 2*y
    # B = np.zeros(2)
    # C = -2*r
    # nu = np.sqrt(np.sum((y-x)**2)/(4*np.sum(y**2)))
    # H =(1-2*nu) *np.eye(2)
    return r*y/(np.sum(y*y))

# modified from DDN basic node
fY = grad(f, 1)
hY = grad(h,1)
hR = grad(h,0)
frY = jacobian(fY, 0)
fYY = jacobian(fY, 1)
hYY = jacobian(hY, 1)
hrY= jacobian(hY, 0)
def gradient_by_auto_diff(r,x):
    y= solve_analyical(x,r)
    indx = np.nonzero(hY(r, y))
    if len(indx[0]) == 0:
        nu= 0.0
    nu = fY(r, y, x)[indx[0][0]] / hY(r, y)[indx[0][0]]
    H = fYY(r, y, x) - nu * hYY(r, y)
    a = hY(r, y)
    B = frY(r, y, x) - nu * hrY(r, y)
    C = hR(r, y)
    con = np.stack((a, B), axis=1)
    try:
        v = sci.linalg.solve(H, con, assume_a='pos')
    except:
         return np.full((2, 1), np.nan).squeeze()
    return (np.outer(v[:, 0], (v[:, 0].dot(B) - C) / v[:, 0].dot(a)) - v[:, 1:1 + 1]).squeeze()

y1= solve_analyical(x, r)
print("forward y error betwen analyical and opt minimize", np.abs(np.sum(y1- solve_opt(x,r,f))))
print("gradient error beween closed-form and autograd:",np.abs(np.sum(gradient_closed_form(r_init,y1,x)-gradient_by_auto_diff(r_init,x))))
print("gradient error beween closed-form and DDN analyical:",np.abs(np.sum(gradient_analyic(r_init,x)-gradient_closed_form(r_init,y1,x))))


# fixed x, change radius, view y(x,r) and Dy(r).
As we can see, the derivative is only related to x, same conclusion from closed form gradient. 
$$
\begin{array}{lll}
    y(r) = \frac {r} {\| x\|_2} x\\
    Dy(r) = \frac {1} {\| x\|_2} x\\
\end{array}
$$

In [ ]:
x = np.random.rand(2)
rs = np.linspace(0.25, 2.25, 101)
ys = [solve_opt(x,r,f) for r in rs]

In [ ]:

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(rs,ys)
plt.grid()
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')

plt.subplot(2, 1, 2)
plt.plot(rs, [gradient_by_auto_diff(r,x) for r in rs])
#plt.plot(x, [gradient_by_ift(xi, yi) for xi, yi in zip(x, y)])
plt.xlabel(r'$x$'); plt.ylabel(r'$dy/dx$')
plt.grid()

plt.show()

# fix radius, change x = (1, x2) , view y(x,r) and Dy(r).
As we can see, y increases, $dy_2$ increase, $dy_1$ increase, however $\|dy \| $ still the same.
$$
\begin{array}{lll}
    y(r) = \frac {r} {\| x\|_2} x\\
    Dy(r) = \frac {1} {\| x\|_2} x\\
\end{array}
$$

In [ ]:
x2 = np.linspace(0.25, 2.25, 101)
x1 = np.ones(101)
xs = np.stack((x1, x2)).T
r = random.uniform(0.1, 10)
ys = [solve_analyical(x,r) for x in xs]
ys2 = [solve_opt(x,r,f) for x in xs][:5]

In [ ]:
plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,ys)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')

plt.subplot(4, 1, 2)
plt.plot(x2, [gradient_by_auto_diff(r,x) for x in xs])
plt.xlabel(r'$x$'); plt.ylabel(r'$dy/dx$')
plt.legend([r"$dy_1$", r"$dy_2$"])
plt.grid()

plt.subplot(4, 1, 3)
plt.plot(x2, [np.linalg.norm(y) for y in ys])
plt.xlabel(r'$x$'); plt.ylabel(r'$dy/dx$')
plt.legend([r"$\|y\|$",])
plt.grid()

plt.subplot(4, 1, 4)
plt.plot(x2, [np.linalg.norm(gradient_by_auto_diff(r,x)) for x in xs])
plt.xlabel(r'$x$'); plt.ylabel(r'$dy/dx$')
plt.legend([r"$\|dy\|$",])
plt.grid()

plt.show()

# Compare y(x,r), Dy(x,r) using different Ln norm.

In [ ]:
def f(r,y,x):
    return 0.5* np.dot(y-x,y-x)
    
def h(r,y,norm):
    if norm=='L1':
       # print(np.linalg.norm(y, 1)- np.sum(np.abs(y)))
        return np.sum(np.abs(y))-r
    if norm=='Ln':
        # print(np.max(np.abs(y))-np.linalg.norm(y, np.inf))
        return np.max(np.abs(y)) - r
    elif norm=='L2':
        return np.dot(y,y) - r**2
def solve_opt(x,r,f,norm):
    result = opt.minimize(lambda y: f(r, y ,x), np.array([1.0,1.0]),constraints=[{'type':'eq', 'fun': lambda y: h(r,y,norm)}] )
    return result.x

fY = grad(f, 1)
hY = grad(h,1)
hR = grad(h,0)
frY = jacobian(fY, 0)
fYY = jacobian(fY, 1)
hYY = jacobian(hY, 1)
hrY= jacobian(hY, 0)
def gradient_by_auto_diff(r,x,norm):
    y= solve_opt(x,r,f,norm)
    #y=solve_analyical(x,r)
    # print(y-solve_analyical(x,r))
    indx = np.nonzero(hY(r, y,norm))
    if len(indx[0]) == 0:
        nu= 0.0
    nu = fY(r, y, x)[indx[0][0]] / hY(r, y, norm)[indx[0][0]]
    H = fYY(r, y, x) - nu * hYY(r, y, norm)
    a = hY(r, y, norm)
    B = frY(r, y, x) - nu * hrY(r, y, norm)
    C = hR(r, y, norm)
    con = np.stack((a, B), axis=1)
    try:
        v = sci.linalg.solve(H, con, assume_a='pos')
    except:
         return np.full((2, 1), np.nan).squeeze()
    #print(nu,fYY(r, y, x),hYY(r, y, norm))
    return (np.outer(v[:, 0], (v[:, 0].dot(B) - C) / v[:, 0].dot(a)) - v[:, 1:1 + 1]).squeeze()
def gradient_L1(r,x):
    y= solve_opt(x,r,f,'L1')
    a = np.sign(y)
    nu= (y-x)[0]*a[0]
    B = np.zeros(2)- nu*np.zeros(2)
    C = -1.0
    H = np.eye(2)-nu*np.zeros(2)
    return a/(a@a)

def gradient_Ln(r,x):
    y= solve_opt(x,r,f,'Ln')
    a = np.array([0 if np.abs(yi)<np.max(np.abs(y)) else np.sign(yi) for yi in y])
    idx= np.where(y==y[np.abs(y)>=np.max(np.abs(y))])
    nu=np.sign(y)[idx]* (y-x)[idx]
    B= np.zeros(2)- nu*np.zeros(2)
    C= -1.0
    H = np.eye(2)-nu*np.zeros(2)
    return a/(a@a)

In [ ]:
x2 = np.linspace(-2.25, 2.25, 101)
x1 = np.ones(101)
xs = np.stack((x1, x2)).T
r = random.uniform(0.1, 10)


In [ ]:
ys1 = [solve_opt(x,r,f,'L1') for x in xs]
ys2 = [solve_opt(x,r,f,'L2') for x in xs]
ys3 = [solve_opt(x,r,f,'Ln') for x in xs]
dys1 = [gradient_by_auto_diff(r,x,'L1') for x in xs]
dys2 = [gradient_by_auto_diff(r,x,'L2') for x in xs]
dys22 = [gradient_analyic(r,x) for x in xs]
dys3 = [gradient_by_auto_diff(r,x,'Ln') for x in xs]

In [ ]:
plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,ys1)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')

plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,ys2)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')

plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,ys3)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')

In [ ]:
plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,dys1)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')


plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,dys2)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')


plt.figure()
plt.subplot(4, 1, 1)
plt.plot(x2,dys3)
plt.grid()
plt.legend([r"$y_1$", r"$y_2$"])
plt.title(r'$y = argmin_u f(x, u)$'); plt.ylabel(r'$y$')


In [ ]:
# b'ABNORMAL_TERMINATION_IN_LNSRCH'
# l1 ln h(r) 
# todo
# what need to show in the tutorial
# verify the gradient with L1, Ln norm, currently the behavior is not clear(in theory I only changed the f function)
# further check/debug gradient descent in projection and pooling
# train both parameter
# possible more parameteric ddn node worth training.